In [4]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 30.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 15.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system 

In [5]:
### import our model
from transformers import AutoModelForMaskedLM
model_id='distilbert-base-uncased'
model=AutoModelForMaskedLM.from_pretrained(model_id)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

In [6]:
#### how many parameters this model have
distilbert_num_parameters = model.num_parameters() / 1_000_000
print(f"'>>> DistilBERT number of parameters: {round(distilbert_num_parameters)}M'")
print(f"'>>> BERT number of parameters: 110M'")

'>>> DistilBERT number of parameters: 67M'
'>>> BERT number of parameters: 110M'


In [7]:
text = "This is a great [MASK]."

In [8]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_id)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [9]:
import torch
inputs=tokenizer(text,return_tensors='pt')
token_logits=model(**inputs).logits
masked_token_id=torch.where(inputs['input_ids']==tokenizer.mask_token_id)[1]
mask_token_logits = token_logits[0, masked_token_id, :]
# Pick the [MASK] candidates with the highest logits
top_5_tokens = torch.topk(mask_token_logits, 5, dim=1).indices[0].tolist()

for token in top_5_tokens:
    print(f"'>>> {text.replace(tokenizer.mask_token, tokenizer.decode([token]))}'")

'>>> This is a great deal.'
'>>> This is a great success.'
'>>> This is a great adventure.'
'>>> This is a great idea.'
'>>> This is a great feat.'


In [10]:
tokenizer.mask_token_id

103

In [11]:
masked_token_id

tensor([5])

In [12]:
token_logits[0,masked_token_id,:]

tensor([[-4.8228, -4.6268, -5.1041,  ..., -4.2771, -5.0184, -3.9428]],
       grad_fn=<IndexBackward0>)

In [13]:
from datasets import load_dataset
raw_dataset=load_dataset('imdb')

README.md:   0%|          | 0.00/7.81k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [14]:
raw_dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

In [15]:
sample=raw_dataset['train'].shuffle(seed=42).select(range(3))
for row in sample:
  print(row['text'])

There is no relation at all between Fortier and Profiler but the fact that both are police series about violent crimes. Profiler looks crispy, Fortier looks classic. Profiler plots are quite simple. Fortier's plot are far more complicated... Fortier looks more like Prime Suspect, if we have to spot similarities... The main character is weak and weirdo, but have "clairvoyance". People like to compare, to judge, to evaluate. How about just enjoying? Funny thing too, people writing Fortier looks American but, on the other hand, arguing they prefer American series (!!!). Maybe it's the language, or the spirit, but I think this series is more English than American. By the way, the actors are really good and funny. The acting is not superficial at all...
This movie is a great. The plot is very true to the book which is a classic written by Mark Twain. The movie starts of with a scene where Hank sings a song with a bunch of kids called "when you stub your toe on the moon" It reminds me of Sin

In [16]:
import pandas as pd
df=pd.DataFrame(raw_dataset['unsupervised'])
df.describe()

,label
count,50000.0
mean,-1.0
std,0.0
min,-1.0
25%,-1.0
50%,-1.0
75%,-1.0
max,-1.0


In [17]:
df.label.unique()## we saw that the unsupervised doesn't have labels

array([-1])

In [18]:
###preprocessing data
def tokenize_function(examples):
  result=tokenizer(examples['text'])
  if tokenizer.is_fast:
    result['word_ids']=[result.word_ids(i) for i in range(len(result['input_ids']))]
  return result

In [19]:
tokenized_datasets=raw_dataset.map(tokenize_function,
                                   batched=True,
                                   remove_columns=['text','label'])

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (720 > 512). Running this sequence through the model will result in indexing errors


Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [20]:
tokenizer.model_max_length## maximum sequence length for model
# we use this for cunk the tokens

512

In [22]:
chunk_size=128

In [39]:
### Let's see some len of our tokens in our tokenized dataset
tokenize_sample=tokenized_datasets['train'][:3]
for idx ,sample in enumerate(tokenize_sample['input_ids']):
  print(f"'>>> Review {idx} length: {len(sample)}'")

'>>> Review 0 length: 363'
'>>> Review 1 length: 304'
'>>> Review 2 length: 133'


In [40]:
concatenated_examples={
    k:sum(tokenize_sample[k],[]) for k in tokenize_sample.keys()
}
total_length = len(concatenated_examples["input_ids"])
print(f"'>>> Concatenated reviews length: {total_length}'")

'>>> Concatenated reviews length: 800'


In [43]:
chunks={
    k:[t[i:i+chunk_size] for i in range(0,total_length,chunk_size)]
    for k,t in concatenated_examples.items()
}

In [45]:
for chunk in chunks['input_ids']:
  print(f"'>>> Chunk length: {len(chunk)}'")

'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 32'


In [53]:
def group_texts(examples):
  concatenated_examples={
      k:sum(examples[k],[])for k in example.keys()
  }
  total_length=len(concatenated_examples[list(examples.keys())[0]])
  total_length=(total_length//chunl_size)*chunk_size
  result={
      k:[t[i:i+chun_size] for i in range(0,total_length,chunk_size)] for k , t in concatenated_examples.items()
  }
  result['labels']=result['input_ids'].copy()
  return result

In [54]:
lm_datasets = tokenized_datasets.map(group_texts, batched=True)
lm_datasets

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 61291
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 59904
    })
    unsupervised: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 122957
    })
})

In [56]:
tokenizer.decode(lm_datasets["train"][1]["labels"])

"as the vietnam war and race issues in the united states. in between asking politicians and ordinary denizens of stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men. < br / > < br / > what kills me about i am curious - yellow is that 40 years ago, this was considered pornographic. really, the sex and nudity scenes are few and far between, even then it ' s not shot like some cheaply made porno. while my countrymen mind find it shocking, in reality sex and nudity are a major staple in swedish cinema. even ingmar bergman,"

In [72]:
from transformers import DataCollatorForLanguageModeling
data_collator=DataCollatorForLanguageModeling(tokenizer=tokenizer,mlm_probability=.15)

In [73]:
samples=[lm_datasets['train'][i] for i in range(2)]
for sample in samples:
  _=sample.pop('word_ids')
for chunk in data_collator(samples)['input_ids']:
   print(f"\n'>>> {tokenizer.convert_ids_to_tokens(chunk)}'")


'>>> ['[CLS]', 'i', 'rented', 'i', 'am', 'curious', '-', 'yellow', 'from', 'my', 'video', 'store', 'because', '[MASK]', 'all', 'the', 'controversy', 'woods', 'surrounded', 'it', 'when', 'it', 'was', 'first', 'released', 'in', '[MASK]', '.', 'i', 'also', 'heard', '[MASK]', 'at', 'first', 'it', 'was', 'seized', 'by', 'u', '.', 's', '.', 'customs', 'if', 'it', 'ever', 'tried', '[MASK]', 'enter', 'this', 'country', ',', 'therefore', 'being', 'a', 'fan', 'of', 'films', 'considered', '"', 'controversial', '[MASK]', 'i', 'really', 'had', 'to', 'see', 'this', 'for', 'myself', '.', '[MASK]', 'br', '/', '>', '<', 'br', '/', '>', 'the', 'plot', 'is', 'centered', '[MASK]', 'a', 'young', 'swedish', 'drama', 'student', 'named', 'lena', 'who', 'wants', 'to', 'learn', 'everything', 'she', 'can', 'about', 'life', '.', 'in', 'particular', 'she', 'wants', 'to', 'focus', '[MASK]', 'attention', '##s', '[MASK]', 'making', 'some', 'sort', 'of', '[MASK]', 'on', 'what', 'the', 'average', 'sw', '##ede', 'thoug

In [74]:
import collections
import numpy as np

from transformers import default_data_collator

wwm_probability = 0.2


def whole_word_masking_data_collator(features):
    for feature in features:
        word_ids = feature.pop("word_ids")

        # Create a map between words and corresponding token indices
        mapping = collections.defaultdict(list)
        current_word_index = -1
        current_word = None
        for idx, word_id in enumerate(word_ids):
            if word_id is not None:
                if word_id != current_word:
                    current_word = word_id
                    current_word_index += 1
                mapping[current_word_index].append(idx)

        # Randomly mask words
        mask = np.random.binomial(1, wwm_probability, (len(mapping),))
        input_ids = feature["input_ids"]
        labels = feature["labels"]
        new_labels = [-100] * len(labels)
        for word_id in np.where(mask)[0]:
            word_id = word_id.item()
            for idx in mapping[word_id]:
                new_labels[idx] = labels[idx]
                input_ids[idx] = tokenizer.mask_token_id
        feature["labels"] = new_labels

    return default_data_collator(features)

In [75]:
samples = [lm_datasets["train"][i] for i in range(2)]
batch = whole_word_masking_data_collator(samples)

for chunk in batch["input_ids"]:
    print(f"\n'>>> {tokenizer.decode(chunk)}'")


'>>> [CLS] i rented [MASK] am curious - yellow from [MASK] video store because of all the controversy that surrounded it when [MASK] was first released [MASK] 1967. i also heard that [MASK] first it was seized by [MASK]. s. [MASK] if it ever tried to enter this [MASK], therefore being a fan of films considered " controversial " i really had to see this for myself. < br / > [MASK] br / > the plot is centered around a young swedish drama student named lena who wants to learn [MASK] she can about life. in particular she [MASK] to focus her attentions to making some sort of documentary [MASK] what the average swede [MASK] about certain [MASK] issues such'

'>>> as the vietnam war and race [MASK] in the united states. in between asking politicians and ordinary denizens of [MASK] [MASK] their opinions [MASK] politics [MASK] she has sex with her drama [MASK], classmates, and married men. < br / [MASK] [MASK] br / > what [MASK] [MASK] about i [MASK] curious [MASK] yellow is that 40 [MASK] [MA

In [76]:
from huggingface_hub import notebook_login

notebook_login()

In [79]:
from transformers import TrainingArguments

batch_size = 64
# Show the training loss with every epoch
logging_steps = len(lm_datasets["train"]) // batch_size
model_name = model_id.split("/")[-1]

training_args = TrainingArguments(
    output_dir=f"{model_name}-finetuned-imdb",
    overwrite_output_dir=True,

    learning_rate=2e-5,
    weight_decay=0.01,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    push_to_hub=True,
    fp16=True,
    logging_steps=logging_steps,
)

In [89]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_datasets["train"],
    eval_dataset=lm_datasets["test"],
    data_collator=whole_word_masking_data_collator,
    tokenizer=tokenizer,
)
trainer.train()

<ipython-input-89-8eb135e9996c>:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


KeyError: 'word_ids'

In [85]:
eval_results = trainer.evaluate()


NameError: name 'math' is not defined

In [86]:
import math
print(f">>> Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

>>> Perplexity: 9.51


In [87]:
trainer.push_to_hub()

events.out.tfevents.1746342338.165c61760ba0.1549.4:   0%|          | 0.00/359 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Alireza0017/distilbert-base-uncased-finetuned-imdb/commit/b4a293d5e67225d50f8c3c78a995d79038c5c362', commit_message='End of training', commit_description='', oid='b4a293d5e67225d50f8c3c78a995d79038c5c362', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Alireza0017/distilbert-base-uncased-finetuned-imdb', endpoint='https://huggingface.co', repo_type='model', repo_id='Alireza0017/distilbert-base-uncased-finetuned-imdb'), pr_revision=None, pr_num=None)